# tdms原始数据读取 
## 不要随便运行， 注释掉了，要用再解除注释

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/High_similar_ML/data_process")
sys.path.insert(0, str(PROJECT_ROOT))

print("📌 Project root set to:", PROJECT_ROOT)


from my_lib.KAIST.KAIST_utils import process_kaist_file
from pathlib import Path

# =========================
# 1. 原始 TDMS 文件列表
# =========================
tdms_files = [
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_03.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_10.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFI_30.tdms",

    # "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms",
    # "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms",
    # "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_30.tdms",

    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_01.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_03.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Misalign_05.tdms",

    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Normal.tdms",

    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_0583mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_1169mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_1751mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_2239mg.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_Unbalance_3318mg.tdms",
]


# =========================
# 2. 保存目录（Linux 侧）
# =========================
save_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
save_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 3. 批量处理
# =========================
saved_csv_files = []

for tdms_path in tdms_files:
    print(f"🚀 Processing: {tdms_path}")
    out_csv = process_kaist_file(
        file_path=tdms_path,
        save_dir=str(save_dir),
        data_type="temperature_current"
    )
    saved_csv_files.append(out_csv)
    print(f"✅ Saved to: {out_csv}")

print(f"\n🎉 Done! Total files processed: {len(saved_csv_files)}")



📌 Project root set to: /home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/High_similar_ML/data_process
🚀 Processing: /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms
✅ Saved to: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_03.csv
🚀 Processing: /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms
✅ Saved to: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_10.csv
🚀 Processing: /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_30.tdms
✅ Saved to: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0N

### 针对BPFO数据进行单独处理，这些数据应该是所有时间点都有缺失，所以用上面的逻辑提取出来是空文件

In [10]:
# ============================================================
# ⭐ 单 Cell：KAIST TDMS → CSV（current & temp）
#    - 自动识别 Log 组
#    - 所有通道对齐长度
#    - 空值统一填 null
# ============================================================

from nptdms import TdmsFile
import pandas as pd
import numpy as np
from pathlib import Path

# =========================
# 1️⃣ TDMS 文件列表
# =========================
tdms_files = [
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms",
    "/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_30.tdms",
]

# =========================
# 2️⃣ 输出目录
# =========================
save_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
save_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 3️⃣ 转换函数（核心）
# =========================
def tdms_to_csv(tdms_path, save_dir):
    tdms = TdmsFile.read(tdms_path)
    log_group = tdms["Log"]

    channel_data = {}
    max_len = 0

    # 读取所有 channel
    for ch in log_group.channels():
        data = ch[:]
        name = ch.name.replace("/", "_")  # 列名安全化

        if len(data) > 0:
            channel_data[name] = data
            max_len = max(max_len, len(data))
            print(f"✅ {name}: {len(data)} samples")
        else:
            channel_data[name] = []  # 空通道
            print(f"⚠️ {name}: empty")

    # 对齐长度，空的 / 不足的补 NaN
    aligned_data = {}
    for name, data in channel_data.items():
        if len(data) == 0:
            aligned_data[name] = [np.nan] * max_len
        elif len(data) < max_len:
            aligned_data[name] = np.pad(
                data,
                (0, max_len - len(data)),
                constant_values=np.nan
            )
        else:
            aligned_data[name] = data

    df = pd.DataFrame(aligned_data)

    # 文件名作为 label
    label = Path(tdms_path).stem
    df["label"] = label

    # 保存
    out_csv = save_dir / f"{label}.csv"
    df.to_csv(out_csv, index=False, na_rep="null")

    print(f"💾 Saved: {out_csv} | shape={df.shape}")
    return out_csv

# =========================
# 4️⃣ 批量执行
# =========================
saved_files = []

for tdms_path in tdms_files:
    print("=" * 60)
    print(f"🚀 Processing {tdms_path}")
    out = tdms_to_csv(tdms_path, save_dir)
    saved_files.append(out)

print("\n🎉 All done!")
print("Generated CSV files:")
for f in saved_files:
    print(" -", f)


🚀 Processing /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms
✅ cDAQ9185-1F486B5Mod1_ai0: 1536492 samples
✅ cDAQ9185-1F486B5Mod1_ai1: 1536492 samples
✅ cDAQ9185-1F486B5Mod2_ai0: 1536492 samples
⚠️ cDAQ9185-1F486B5Mod2_ai2: empty
⚠️ cDAQ9185-1F486B5Mod2_ai3: empty
💾 Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPFO_03.csv | shape=(1536492, 6)
🚀 Processing /home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_10.tdms
✅ cDAQ9185-1F486B5Mod1_ai0: 1536492 samples
✅ cDAQ9185-1F486B5Mod1_ai1: 1536492 samples
✅ cDAQ9185-1F486B5Mod2_ai0: 1536492 samples
⚠️ cDAQ9185-1F486B5Mod2_ai2: empty
⚠️ cDAQ9185-1F486B5Mod2_ai3: empty
💾 Saved: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw/0Nm_BPF

## 重写列名

In [ ]:
from pathlib import Path
import pandas as pd

# =========================
# 1. 输入 / 输出目录
# =========================
src_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw")
dst_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")
dst_dir.mkdir(parents=True, exist_ok=True)

# =========================
# 2. 原始 → 标准列名映射（核心）
# =========================
column_rename_map = {
    "timestamp": "timestamp",

    "cDAQ9185-1F486B5Mod1/ai0": "Temp_A(°C)",
    "cDAQ9185-1F486B5Mod1/ai1": "Temp_B(°C)",

    "cDAQ9185-1F486B5Mod2/ai0": "I_U(A)",
    "cDAQ9185-1F486B5Mod2/ai2": "I_V(A)",
    "cDAQ9185-1F486B5Mod2/ai3": "I_W(A)",
}

expected_cols = set(column_rename_map.values())

# =========================
# 3. 批量处理
# =========================
csv_files = sorted(src_dir.glob("*.csv"))
print(f"📂 Found {len(csv_files)} CSV files")

for csv_path in csv_files:
    print(f"\n🔄 Processing: {csv_path.name}")

    df = pd.read_csv(csv_path)

    # 校验：是否包含所有原始列
    missing_raw = set(column_rename_map.keys()) - set(df.columns)
    if missing_raw:
        raise ValueError(
            f"{csv_path.name} 缺少原始列: {missing_raw}"
        )

    # 重命名
    df = df.rename(columns=column_rename_map)

    # 校验：重命名后是否齐全
    missing_after = expected_cols - set(df.columns)
    if missing_after:
        raise ValueError(
            f"{csv_path.name} 重命名后缺少列: {missing_after}"
        )

    # 保存
    out_path = dst_dir / csv_path.name
    df.to_csv(out_path, index=False)

    print(f"✅ Saved to: {out_path}")

print("\n🎉 所有文件列名已统一并成功保存。")


## 加入标签

In [ ]:
from pathlib import Path
import pandas as pd

# =========================
# 1. 数据目录
# =========================
data_dir = Path("/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel")

# =========================
# 2. 文件名 → 标签 映射表
# =========================
label_map = {
    "0Nm_Normal": 1,

    "0Nm_BPFI_03": 2,
    "0Nm_BPFI_10": 3,
    "0Nm_BPFI_30": 4,

    "0Nm_BPFO_03": 5,
    "0Nm_BPFO_10": 6,
    "0Nm_BPFO_30": 7,

    "0Nm_Misalign_01": 8,
    "0Nm_Misalign_03": 9,
    "0Nm_Misalign_05": 10,

    "0Nm_Unbalance_0583mg": 11,
    "0Nm_Unbalance_1169mg": 12,
    "0Nm_Unbalance_1751mg": 13,
    "0Nm_Unbalance_2239mg": 14,
    "0Nm_Unbalance_3318mg": 15,
}

# =========================
# 3. 批量处理
# =========================
csv_files = sorted(data_dir.glob("*.csv"))
print(f"📂 Found {len(csv_files)} CSV files")

for csv_path in csv_files:
    stem = csv_path.stem   # 去掉 .csv
    print(f"\n🔄 Processing: {csv_path.name}")

    if stem not in label_map:
        raise ValueError(f"❌ 文件名未在映射表中: {csv_path.name}")

    label = label_map[stem]

    df = pd.read_csv(csv_path)

    # 新增 label 列（放在最后，最稳）
    df["label"] = label

    # 覆盖保存
    df.to_csv(csv_path, index=False)

    print(f"✅ Added label={label} to {csv_path.name}")

print("\n🎉 所有文件已成功添加 label 列。")


## 提取每个样本5000个

In [7]:
import os
import glob
import pandas as pd

# ==========================
# 1️⃣ 参数区（你只需要改这里）
# ==========================
data_dir = "/home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel"
output_csv = os.path.join(data_dir, "0Nm_all_classes_merged.csv")

samples_per_class = 5000      # ⭐ 每个类别抽取样本数（可调）
shuffle_each_class = True     # 是否在类别内随机抽样
random_state = 42             # 保证可复现

# ==========================
# 2️⃣ 读取 & 抽样
# ==========================
all_dfs = []

csv_files = sorted(glob.glob(os.path.join(data_dir, "*.csv")))

assert len(csv_files) > 0, "❌ 文件夹下没有 CSV 文件"

print(f"🔍 共发现 {len(csv_files)} 个类别文件")

for csv_path in csv_files:
    class_name = os.path.basename(csv_path).replace(".csv", "")
    df = pd.read_csv(csv_path)

    # --------------------------
    # 如果没有 label 列，用文件名生成
    # --------------------------
    if "label" not in df.columns:
        df["label"] = class_name

    # --------------------------
    # 抽样
    # --------------------------
    if len(df) >= samples_per_class:
        df_sampled = df.sample(
            n=samples_per_class,
            replace=False,
            random_state=random_state if shuffle_each_class else None
        )
    else:
        print(f"⚠️ {class_name} 样本不足 {samples_per_class}，仅有 {len(df)}，已全部使用")
        df_sampled = df

    df_sampled["source_file"] = class_name  # 可选：保留来源
    all_dfs.append(df_sampled)

    print(f"✅ {class_name}: {len(df_sampled)} samples")

# ==========================
# 3️⃣ 合并 & 保存
# ==========================
merged_df = pd.concat(all_dfs, ignore_index=True)

merged_df.to_csv(output_csv, index=False)

print("=" * 60)
print("🎉 合并完成")
print(f"📁 输出路径: {output_csv}")
print(f"📊 总样本数: {len(merged_df)}")
print("📈 各类别样本统计：")
print(merged_df["label"].value_counts())


🔍 共发现 15 个类别文件
✅ 0Nm_BPFI_03: 5000 samples
✅ 0Nm_BPFI_10: 5000 samples
✅ 0Nm_BPFI_30: 5000 samples
⚠️ 0Nm_BPFO_03 样本不足 5000，仅有 0，已全部使用
✅ 0Nm_BPFO_03: 0 samples
⚠️ 0Nm_BPFO_10 样本不足 5000，仅有 0，已全部使用
✅ 0Nm_BPFO_10: 0 samples
⚠️ 0Nm_BPFO_30 样本不足 5000，仅有 0，已全部使用
✅ 0Nm_BPFO_30: 0 samples
✅ 0Nm_Misalign_01: 5000 samples
✅ 0Nm_Misalign_03: 5000 samples
✅ 0Nm_Misalign_05: 5000 samples
✅ 0Nm_Normal: 5000 samples
✅ 0Nm_Unbalance_0583mg: 5000 samples
✅ 0Nm_Unbalance_1169mg: 5000 samples
✅ 0Nm_Unbalance_1751mg: 5000 samples
✅ 0Nm_Unbalance_2239mg: 5000 samples
✅ 0Nm_Unbalance_3318mg: 5000 samples


/tmp/ipykernel_113468/1363209527.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(all_dfs, ignore_index=True)


🎉 合并完成
📁 输出路径: /home/charles/HZU/Data_processed/HSML/KAIST/current&temp_raw_withlabel/0Nm_all_classes_merged.csv
📊 总样本数: 60000
📈 各类别样本统计：
label
2     5000
3     5000
4     5000
8     5000
9     5000
10    5000
1     5000
11    5000
12    5000
13    5000
14    5000
15    5000
Name: count, dtype: int64


In [8]:
from nptdms import TdmsFile

tdms = TdmsFile.read("/home/charles/HZU/Data_raw/KAIST/Vibration, Acoustic, Temperature, and Motor Current Dataset of Rotating Machine Under Varying Load Conditions for Fault Diagnosis/current,temp/0Nm_BPFO_03.tdms")

# 查看结构
print(tdms.groups())

for group in tdms.groups():
    print("Group:", group.name)
    for channel in group.channels():
        print("  Channel:", channel.name, channel[:10])


[<TdmsGroup with path /'Test Information'>, <TdmsGroup with path /'Log'>]
Group: Test Information
Group: Log
  Channel: cDAQ9185-1F486B5Mod1/ai0 [28.99347175 28.99347175 28.99347175 28.99347175 28.99347175 28.99347175
 28.99347175 28.99347175 28.99347175 28.99347175]
  Channel: cDAQ9185-1F486B5Mod1/ai1 [29.99540662 29.99540662 29.99540662 29.99540662 29.99540662 29.99540662
 29.99540662 29.99540662 29.99540662 29.99540662]
  Channel: cDAQ9185-1F486B5Mod2/ai0 [-1.18582077 -1.13782124 -0.94582314 -0.9513088  -0.93073758 -1.18582077
 -1.13919265 -0.91153777 -0.71953967 -1.02947946]
  Channel: cDAQ9185-1F486B5Mod2/ai2 []
  Channel: cDAQ9185-1F486B5Mod2/ai3 []
